In [1]:
import tensorflow as tf # http://blog.topspeedsnail.com/archives/10399
from sklearn.preprocessing import scale # 使用scikit-learn进行数据预处理
import pandas as pd
import numpy as np

In [2]:
training_data = pd.read_csv('./UJIndoorLoc/trainingData.csv', header = 0)
#print(training_data.head())
train_x = scale(np.asarray(training_data.ix[:, 0:520]))
train_y = np.asarray(training_data['BUILDINGID'].map(str) + training_data['FLOOR'].map(str))
train_y = np.asarray(pd.get_dummies(train_y))

In [3]:
test_dataset = pd.read_csv('./UJIndoorLoc/validationData.csv', header = 0)
test_x = scale(np.asarray(test_dataset.ix[:, 0:520]))
test_y = np.asarray(test_dataset['BUILDINGID'].map(str) + test_dataset['FLOOR'].map(str))
test_y = np.asarray(pd.get_dummies(test_y))

In [4]:
output = train_y.shape[1]
X = tf.placeholder(tf.float32, shape=[None, 520])
Y = tf.placeholder(tf.float32, [None, output])

In [5]:
# 定义神经网络
def neural_networks():
    # --------------------- Encoder -------------------- #
    e_w_1 = tf.Variable(tf.truncated_normal([520, 256], stddev = 0.1))
    e_b_1 = tf.Variable(tf.constant(0.0, shape=[256]))
    e_w_2 = tf.Variable(tf.truncated_normal([256, 128], stddev = 0.1))
    e_b_2 = tf.Variable(tf.constant(0.0, shape=[128]))
    e_w_3 = tf.Variable(tf.truncated_normal([128, 64], stddev = 0.1))
    e_b_3 = tf.Variable(tf.constant(0.0, shape=[64]))
    # --------------------- Decoder  ------------------- #
    d_w_1 = tf.Variable(tf.truncated_normal([64, 128], stddev = 0.1))
    d_b_1 = tf.Variable(tf.constant(0.0, shape=[128]))
    d_w_2 = tf.Variable(tf.truncated_normal([128, 256], stddev = 0.1))
    d_b_2 = tf.Variable(tf.constant(0.0, shape=[256]))
    d_w_3 = tf.Variable(tf.truncated_normal([256, 520], stddev = 0.1))
    d_b_3 = tf.Variable(tf.constant(0.0, shape=[520]))
    # --------------------- DNN  ------------------- #
    w_1 = tf.Variable(tf.truncated_normal([64, 128], stddev = 0.1))
    b_1 = tf.Variable(tf.constant(0.0, shape=[128]))
    w_2 = tf.Variable(tf.truncated_normal([128, 128], stddev = 0.1))
    b_2 = tf.Variable(tf.constant(0.0, shape=[128]))
    w_3 = tf.Variable(tf.truncated_normal([128, output], stddev = 0.1))
    b_3 = tf.Variable(tf.constant(0.0, shape=[output]))
    #########################################################
    layer_1 = tf.nn.tanh(tf.add(tf.matmul(X,       e_w_1), e_b_1))
    layer_2 = tf.nn.tanh(tf.add(tf.matmul(layer_1, e_w_2), e_b_2))
    encoded = tf.nn.tanh(tf.add(tf.matmul(layer_2, e_w_3), e_b_3))
    layer_4 = tf.nn.tanh(tf.add(tf.matmul(encoded, d_w_1), d_b_1))
    layer_5 = tf.nn.tanh(tf.add(tf.matmul(layer_4, d_w_2), d_b_2))
    decoded = tf.nn.tanh(tf.add(tf.matmul(layer_5, d_w_3), d_b_3))
    layer_7 = tf.nn.tanh(tf.add(tf.matmul(encoded, w_1),   b_1))
    layer_8 = tf.nn.tanh(tf.add(tf.matmul(layer_7, w_2),   b_2))
    out = tf.nn.softmax(tf.add(tf.matmul( layer_8, w_3),   b_3))
    return (decoded, out)

In [6]:
# 训练神经网络
def train_neural_networks():
    decoded, predict_output = neural_networks()
    
    us_cost_function = tf.reduce_mean(tf.pow(X - decoded, 2))
    s_cost_function = -tf.reduce_sum(Y * tf.log(predict_output))
    us_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(us_cost_function)
    s_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(s_cost_function)
    
    correct_prediction = tf.equal(tf.argmax(predict_output, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    training_epochs = 20
    batch_size = 10
    total_batches = training_data.shape[0]
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        # ------------ Training Autoencoders - Unsupervised Learning ----------- #
        # autoencoder是一种非监督学习算法，他利用反向传播算法，让目标值等于输入值
        for epoch in range(training_epochs):
            epoch_costs = np.empty(0)
            for b in range(total_batches):
                offset = (b * batch_size) % (train_x.shape[0] - batch_size)
                batch_x = train_x[offset:(offset + batch_size), :]
                _, c = sess.run([us_optimizer, us_cost_function], feed_dict={X: batch_x})
                epoch_costs = np.append(epoch_costs, c)
            print('Epoch: ', epoch, ' Loss: ', np.mean(epoch_costs))
        print("------------------------------------------------------------------")
        
        # ---------------- Training NN - Supervised Learning ------------------ #
        for epoch in range(training_epochs):
            epoch_costs = np.empty(0)
            for b in range(total_batches):
                offset = (b * batch_size) % (train_x.shape[0] - batch_size)
                batch_x = train_x[offset:(offset + batch_size), :]
                batch_y = train_y[offset:(offset + batch_size), :]
                _, c = sess.run([s_optimizer, s_cost_function], feed_dict={X:batch_x, Y:batch_y})
                epoch_costs = np.append(epoch_costs, c)
            
            accuracy_in_train_set = sess.run(accuracy, feed_dict={X:train_x, Y:train_y})
            accuracy_in_test_set = sess.run(accuracy, feed_dict={X:test_x, Y:test_y})
            print('Epoch: ', epoch, ' Loss: ', np.mean(epoch_costs), ' Accuracy: ', accuracy_in_train_set, ' ', accuracy_in_test_set)

In [7]:
train_neural_networks()

Epoch:  0  Loss:  0.811163063365
Epoch:  1  Loss:  0.745404697547
Epoch:  2  Loss:  0.730484397802
Epoch:  3  Loss:  0.722523403331
Epoch:  4  Loss:  0.717252474601
Epoch:  5  Loss:  0.713395552334
Epoch:  6  Loss:  0.710352347159
Epoch:  7  Loss:  0.707823287564
Epoch:  8  Loss:  0.705657089711
Epoch:  9  Loss:  0.703769226292
Epoch:  10  Loss:  0.702106146214
Epoch:  11  Loss:  0.700628797791
Epoch:  12  Loss:  0.699305994264
Epoch:  13  Loss:  0.698111438284
Epoch:  14  Loss:  0.697022781142
Epoch:  15  Loss:  0.696022743388
Epoch:  16  Loss:  0.695096936329
Epoch:  17  Loss:  0.694233497918
Epoch:  18  Loss:  0.693423232996
Epoch:  19  Loss:  0.692658683423
------------------------------------------------------------------
Epoch:  0  Loss:  1.39020893846  Accuracy:  0.755279   0.593159
Epoch:  1  Loss:  0.472183999188  Accuracy:  0.911923   0.711971
Epoch:  2  Loss:  0.251755594388  Accuracy:  0.959272   0.718272
Epoch:  3  Loss:  0.199712017755  Accuracy:  0.917691   0.679568
Epoc